Use any dataset of plant disease and design a plant disease detection system using CNN.

Importing Images & Libraries

In [ ]:
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d noulam/tomato

!unzip -q tomato.zip

Saving kaggle.json to kaggle.json
100% 743M/743M [00:36<00:00, 22.5MB/s]
100% 743M/743M [00:36<00:00, 21.4MB/s]


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [ ]:
#https://www.kaggle.com/datasets/noulam/tomato (Link to the dataset)

#Originally New Plant Diseases Dataset(Augmented) contains two folders [train & valid] each containing 10 directories i.e. 10 classes - 1. Tomato___Bacterial_spot 2. Tomato___Early_blight 3. Tomato___Late_blight 4. Tomato___Leaf_Mold 5. Tomato___Septoria_leaf_spot 6. Tomato___Spider_mites Two-spotted_spider_mite 7. Tomato___Target_Spot 8. Tomato___Tomato_Yellow_Leaf_Curl_Virus 9. Tomato___Tomato_mosaic_virus 10. Tomato___healthy [Train - 18345 Total Images, Valid - 4585 Total Images]

#But for training & validating we have taken only three classes namely 1. Tomato___Bacterial_spot 2. Tomato___Early_blight 3. Tomato___healthy each class containing 200 images i.e. total images - 600 [Train - 600 Total Images, Valid - 600 Total Images]

In [ ]:
#Path to the New Plant Diseases Dataset(Augmented)
train_dir = 'New Plant Diseases Dataset(Augmented)/train'
val_dir = 'New Plant Diseases Dataset(Augmented)/valid'

In [ ]:
img_size = 224
batch_size = 32

Preprocessing

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir,target_size=(img_size, img_size),batch_size=batch_size,
                                                    class_mode='categorical')

Found 18345 images belonging to 10 classes.


In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(img_size, img_size),
                                                batch_size=batch_size,
                                                class_mode='categorical')

Found 4585 images belonging to 10 classes.


In [ ]:
list(train_generator.class_indices)

['Tomato___Bacterial_spot',
 'Tomato___Early_blight',
 'Tomato___Late_blight',
 'Tomato___Leaf_Mold',
 'Tomato___Septoria_leaf_spot',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Tomato___Target_Spot',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Tomato___Tomato_mosaic_virus',
 'Tomato___healthy']

Building our Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

In [ ]:
model = Sequential()

model.add((Conv2D(32, (3,3), activation='relu', input_shape=(img_size, img_size, 3))))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))
model.add((Conv2D(64, (3,3), activation='relu')))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))
model.add((Conv2D(64, (3,3), activation='relu')))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))
model.add((Conv2D(128, (3,3), activation='relu')))
model.add(BatchNormalization())
model.add((MaxPooling2D(2,2)))

model.add((Flatten()))

model.add((Dense(128, activation='relu')))
model.add((Dropout(0.2)))
model.add((Dense(64, activation='relu')))
model.add((Dense(train_generator.num_classes, activation='softmax')))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 222, 222, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 109, 109, 64)      256       
 chNormalization)                                                
                                                        

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Training our Model

In [ ]:
model.fit(train_generator, epochs=3, validation_data=val_generator)

Epoch 1/3
574/574 [==============================] - 58s 84ms/step - loss: 1.1461 - accuracy: 0.6460 - val_loss: 1.6573 - val_accuracy: 0.5241
Epoch 2/3
574/574 [==============================] - 47s 82ms/step - loss: 0.6085 - accuracy: 0.8043 - val_loss: 2.0613 - val_accuracy: 0.4848
Epoch 3/3
574/574 [==============================] - 48s 83ms/step - loss: 0.4567 - accuracy: 0.8534 - val_loss: 0.5054 - val_accuracy: 0.8427


Evaluating our Model

In [ ]:
loss, accuracy = model.evaluate(val_generator)
print("Loss :",loss)
print("Accuracy (Test Data) :",accuracy*100)

144/144 [==============================] - 9s 63ms/step - loss: 0.5054 - accuracy: 0.8427
Loss : 0.505395233631134
Accuracy (Test Data) : 84.27481055259705


Testing our Model

In [ ]:
import numpy as np
img_path =r'New Plant Diseases Dataset(Augmented)/valid/Tomato___Early_blight/28d03063-a772-4136-80fd-3bbff0fffa41___RS_Erly.B 7370.JPG'
img = load_img(img_path, target_size=(224, 224))
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.

In [ ]:
prediction = model.predict(img_array)
class_names=['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy']

1/1 [==============================] - 0s 29ms/step


In [ ]:
predicted_class = np.argmax(prediction)
print(prediction)
print(predicted_class)
print('Predicted class:', class_names[predicted_class])

[[9.2648637e-01 5.7940453e-02 1.8187028e-03 1.7453975e-05 2.0669166e-04
  1.0303264e-04 1.2118775e-06 1.3422115e-02 3.0397280e-07 3.6898989e-06]]
0
Predicted class: Tomato___Bacterial_spot
